# Creating the Two and Three Zone Example Data

Transform the TM1 TAZ-based model 25 zone inputs to a two-zone (MAZ and TAZ) and three-zone (MAZ, TAZ, TAP) set of inputs for software development.  

The 25 zones are downtown San Francisco and they are converted to 25 MAZs.  Each TAZ is also converted to a TAP numbered 10000+.  MAZs 1,2,3,4 are small and adjacent and assigned TAZ 2 and TAP 10002.  MAZs 13,14,15 are small and adjacent and assigned TAZ 14 and TAP 10014.  TAZs 1,3,4,13,15 are removed from the final data set.  TAPs 10001,10003,10004,10013,10015 are removed from the final data set.  

This script should work for the full TM1 example as well.

# Create examples

In [1]:
!activitysim create -e example_mtc -d example_two_zone

!activitysim create -e example_mtc -d example_three_zone

copying files from example_mtc...
copied! new project files are in C:\projects\development\activitysim_rsg\notebooks\example_two_zone
copying files from example_mtc...
copied! new project files are in C:\projects\development\activitysim_rsg\notebooks\example_three_zone


# Load libraries

In [2]:
import pandas as pd
import numpy as np
import openmatrix as omx
import os, os.path

# Read inputs and write revised inputs for two_zone

In [3]:
%cd example_two_zone

C:\projects\development\activitysim_rsg\notebooks\example_two_zone


### Convert tazs to mazs and add transit access distance by mode

In [4]:
zones = pd.read_csv("data/land_use.csv")
zones["MAZ"] = zones["ZONE"]
zones["TAZ"] = zones["ZONE"]
zones.loc[zones["MAZ"].isin([1,2,3,4]),"TAZ"] = 2
zones.loc[zones["MAZ"].isin([13,14,15]),"TAZ"] = 14
zones = zones.drop(['ZONE'], axis=1)

shortWalk = 0.333 #the tm1 example assumes this distance for transit access

zones["access_dist_com"] = shortWalk
zones["access_dist_exp"] = shortWalk
zones["access_dist_hvy"] = shortWalk
zones["access_dist_loc"] = shortWalk
zones["access_dist_lrf"] = shortWalk

zones.to_csv("data/land_use.csv", index=False)
zones

,DISTRICT,SD,COUNTY,TOTHH,HHPOP,TOTPOP,EMPRES,SFDU,MFDU,HHINCQ1,...,hhlds,sftaz,gqpop,MAZ,TAZ,access_dist_com,access_dist_exp,access_dist_hvy,access_dist_loc,access_dist_lrf
0,1,1,1,46,74,82,37,1,60,15,...,46,1,8,1,2,0.333,0.333,0.333,0.333,0.333
1,1,1,1,134,214,240,107,5,147,57,...,134,2,26,2,2,0.333,0.333,0.333,0.333,0.333
2,1,1,1,267,427,476,214,9,285,101,...,267,3,49,3,2,0.333,0.333,0.333,0.333,0.333
3,1,1,1,151,239,253,117,6,210,52,...,151,4,14,4,2,0.333,0.333,0.333,0.333,0.333
4,1,1,1,611,974,1069,476,22,671,223,...,611,5,95,5,5,0.333,0.333,0.333,0.333,0.333
5,1,1,1,2240,3311,3963,2052,0,2406,843,...,2240,6,652,6,6,0.333,0.333,0.333,0.333,0.333
6,1,1,1,3762,5561,6032,3375,0,4174,1489,...,3762,7,471,7,7,0.333,0.333,0.333,0.333,0.333
7,1,1,1,4582,7565,9907,3594,19,4898,2704,...,4582,8,2342,8,8,0.333,0.333,0.333,0.333,0.333
8,1,1,1,5545,9494,10171,4672,35,6032,2605,...,5545,9,677,9,9,0.333,0.333,0.333,0.333,0.333
9,1,1,1,5344,9205,9308,5137,5,5663,3825,...,5344,10,103,10,10,0.333,0.333,0.333,0.333,0.333


### Put households in mazs instead of tazs

In [5]:
hhs = pd.read_csv("data/households.csv")
hhs["MAZ"] = hhs["TAZ"]
hhs = hhs.drop(['TAZ'], axis=1)
hhs.to_csv("data/households.csv", index=False)
hhs

,HHID,SERIALNO,PUMA5,income,PERSONS,HHT,UNITTYPE,NOC,BLDGSZ,TENURE,...,hschdriv,htypdwel,hownrent,hadnwst,hadwpst,hadkids,bucketBin,originalPUMA,hmultiunit,MAZ
0,2717868,2715386,2202,361000,2,1,0,0,9,1,...,0,2,1,0,0,0,3,2202,1,25
1,763899,5360279,2203,59220,1,4,0,0,9,3,...,0,2,2,0,0,0,4,2203,1,6
2,2222791,77132,2203,197000,2,2,0,0,9,1,...,0,2,1,0,0,1,5,2203,1,9
3,112477,3286812,2203,2200,1,6,0,0,8,3,...,0,2,2,0,0,0,7,2203,1,17
4,370491,6887183,2203,16500,3,1,0,1,8,3,...,0,2,2,0,0,0,7,2203,1,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,109218,3592966,2203,15000,1,4,0,0,8,3,...,0,2,2,0,0,0,7,2203,1,10
4996,570708,2418140,2202,13100,1,6,0,0,9,3,...,0,2,2,0,0,0,6,2202,1,23
4997,2762199,4016973,2203,0,1,0,2,0,0,0,...,0,2,2,0,0,0,2,2203,1,21
4998,2049372,965334,2203,103000,1,4,0,0,9,3,...,0,2,2,0,0,0,9,2203,1,18


### Create taz file and taz skims

In [6]:
skims_file = omx.open_file("data/skims.omx")
skims = skims_file.list_matrices()
num_zones = skims_file.shape()[0]
zone_labels = np.arange(1,(num_zones+1)) #seq from 1 to X

zones_to_drop = [1,3,4,13,15]
new_zone_labels = np.delete(zone_labels, np.array(zones_to_drop) - 1)
new_zone_indexes = new_zone_labels - 1 

#create taz file
tazs = pd.DataFrame({"taz":new_zone_labels})
tazs.to_csv("data/tazs.csv", index=False)
print(tazs)

output_skims_file = omx.open_file("data/taz_skims.omx", "w")

for a_skim in skims_file.list_matrices():
    
    if not (a_skim == "DISTBIKE" or a_skim == "DISTWALK"): #omit these since using MAZ to MAZ distances
        
        output_skims_file[a_skim] = skims_file[a_skim][:][new_zone_indexes,:][:,new_zone_indexes]
        print("skim:", a_skim, ": shape", str(output_skims_file.shape()))

output_skims_file.create_mapping("taz", new_zone_labels)
    
skims_file.close()
output_skims_file.close()

    taz
0     2
1     5
2     6
3     7
4     8
5     9
6    10
7    11
8    12
9    14
10   16
11   17
12   18
13   19
14   20
15   21
16   22
17   23
18   24
19   25
skim: DIST : shape (20, 20)
skim: DRV_COM_WLK_BOARDS__AM : shape (20, 20)
skim: DRV_COM_WLK_BOARDS__EA : shape (20, 20)
skim: DRV_COM_WLK_BOARDS__EV : shape (20, 20)
skim: DRV_COM_WLK_BOARDS__MD : shape (20, 20)
skim: DRV_COM_WLK_BOARDS__PM : shape (20, 20)
skim: DRV_COM_WLK_DDIST__AM : shape (20, 20)
skim: DRV_COM_WLK_DDIST__EA : shape (20, 20)
skim: DRV_COM_WLK_DDIST__EV : shape (20, 20)
skim: DRV_COM_WLK_DDIST__MD : shape (20, 20)
skim: DRV_COM_WLK_DDIST__PM : shape (20, 20)
skim: DRV_COM_WLK_DTIM__AM : shape (20, 20)
skim: DRV_COM_WLK_DTIM__EA : shape (20, 20)
skim: DRV_COM_WLK_DTIM__EV : shape (20, 20)
skim: DRV_COM_WLK_DTIM__MD : shape (20, 20)
skim: DRV_COM_WLK_DTIM__PM : shape (20, 20)
skim: DRV_COM_WLK_FAR__AM : shape (20, 20)
skim: DRV_COM_WLK_FAR__EA : shape (20, 20)
skim: DRV_COM_WLK_FAR__EV : shape (20, 20)


skim: DRV_LOC_WLK_XWAIT__PM : shape (20, 20)
skim: DRV_LRF_WLK_BOARDS__AM : shape (20, 20)
skim: DRV_LRF_WLK_BOARDS__EA : shape (20, 20)
skim: DRV_LRF_WLK_BOARDS__EV : shape (20, 20)
skim: DRV_LRF_WLK_BOARDS__MD : shape (20, 20)
skim: DRV_LRF_WLK_BOARDS__PM : shape (20, 20)
skim: DRV_LRF_WLK_DDIST__AM : shape (20, 20)
skim: DRV_LRF_WLK_DDIST__EA : shape (20, 20)
skim: DRV_LRF_WLK_DDIST__EV : shape (20, 20)
skim: DRV_LRF_WLK_DDIST__MD : shape (20, 20)
skim: DRV_LRF_WLK_DDIST__PM : shape (20, 20)
skim: DRV_LRF_WLK_DTIM__AM : shape (20, 20)
skim: DRV_LRF_WLK_DTIM__EA : shape (20, 20)
skim: DRV_LRF_WLK_DTIM__EV : shape (20, 20)
skim: DRV_LRF_WLK_DTIM__MD : shape (20, 20)
skim: DRV_LRF_WLK_DTIM__PM : shape (20, 20)
skim: DRV_LRF_WLK_FAR__AM : shape (20, 20)
skim: DRV_LRF_WLK_FAR__EA : shape (20, 20)
skim: DRV_LRF_WLK_FAR__EV : shape (20, 20)
skim: DRV_LRF_WLK_FAR__MD : shape (20, 20)
skim: DRV_LRF_WLK_FAR__PM : shape (20, 20)
skim: DRV_LRF_WLK_FERRYIVT__AM : shape (20, 20)
skim: DRV_LRF_WLK

skim: WLK_COM_WLK_FAR__EA : shape (20, 20)
skim: WLK_COM_WLK_FAR__EV : shape (20, 20)
skim: WLK_COM_WLK_FAR__MD : shape (20, 20)
skim: WLK_COM_WLK_FAR__PM : shape (20, 20)
skim: WLK_COM_WLK_IWAIT__AM : shape (20, 20)
skim: WLK_COM_WLK_IWAIT__EA : shape (20, 20)
skim: WLK_COM_WLK_IWAIT__EV : shape (20, 20)
skim: WLK_COM_WLK_IWAIT__MD : shape (20, 20)
skim: WLK_COM_WLK_IWAIT__PM : shape (20, 20)
skim: WLK_COM_WLK_KEYIVT__AM : shape (20, 20)
skim: WLK_COM_WLK_KEYIVT__EA : shape (20, 20)
skim: WLK_COM_WLK_KEYIVT__EV : shape (20, 20)
skim: WLK_COM_WLK_KEYIVT__MD : shape (20, 20)
skim: WLK_COM_WLK_KEYIVT__PM : shape (20, 20)
skim: WLK_COM_WLK_TOTIVT__AM : shape (20, 20)
skim: WLK_COM_WLK_TOTIVT__EA : shape (20, 20)
skim: WLK_COM_WLK_TOTIVT__EV : shape (20, 20)
skim: WLK_COM_WLK_TOTIVT__MD : shape (20, 20)
skim: WLK_COM_WLK_TOTIVT__PM : shape (20, 20)
skim: WLK_COM_WLK_WAIT__AM : shape (20, 20)
skim: WLK_COM_WLK_WAIT__EA : shape (20, 20)
skim: WLK_COM_WLK_WAIT__EV : shape (20, 20)
skim: WLK_C

skim: WLK_HVY_WLK_WAIT__EA : shape (20, 20)
skim: WLK_HVY_WLK_WAIT__EV : shape (20, 20)
skim: WLK_HVY_WLK_WAIT__MD : shape (20, 20)
skim: WLK_HVY_WLK_WAIT__PM : shape (20, 20)
skim: WLK_HVY_WLK_WAUX__AM : shape (20, 20)
skim: WLK_HVY_WLK_WAUX__EA : shape (20, 20)
skim: WLK_HVY_WLK_WAUX__EV : shape (20, 20)
skim: WLK_HVY_WLK_WAUX__MD : shape (20, 20)
skim: WLK_HVY_WLK_WAUX__PM : shape (20, 20)
skim: WLK_HVY_WLK_XWAIT__AM : shape (20, 20)
skim: WLK_HVY_WLK_XWAIT__EA : shape (20, 20)
skim: WLK_HVY_WLK_XWAIT__EV : shape (20, 20)
skim: WLK_HVY_WLK_XWAIT__MD : shape (20, 20)
skim: WLK_HVY_WLK_XWAIT__PM : shape (20, 20)
skim: WLK_LOC_DRV_BOARDS__AM : shape (20, 20)
skim: WLK_LOC_DRV_BOARDS__EA : shape (20, 20)
skim: WLK_LOC_DRV_BOARDS__EV : shape (20, 20)
skim: WLK_LOC_DRV_BOARDS__MD : shape (20, 20)
skim: WLK_LOC_DRV_BOARDS__PM : shape (20, 20)
skim: WLK_LOC_DRV_DDIST__AM : shape (20, 20)
skim: WLK_LOC_DRV_DDIST__EA : shape (20, 20)
skim: WLK_LOC_DRV_DDIST__EV : shape (20, 20)
skim: WLK_LOC_

### Create maz to maz distance files by mode

In [7]:
skims_file = omx.open_file("data/skims.omx")

distance_table = pd.DataFrame(skims_file['DIST']).unstack()

skims_file.close()

walk_speed = 3
bike_speed = 10
max_distance_for_nearby_mazs_walk = 1.0
max_distance_for_nearby_mazs_bike = 5.0

distance_table = distance_table.reset_index()
distance_table.columns = ["OMAZ","DMAZ","DIST"]
distance_table["OMAZ"] = distance_table["OMAZ"] + 1 #seq from 1 to X
distance_table["DMAZ"] = distance_table["DMAZ"] + 1 #seq from 1 to X

distance_table_walk = distance_table[distance_table["DIST"] <= max_distance_for_nearby_mazs_walk]
distance_table_walk["TIME"] = distance_table["DIST"] * (60 / walk_speed)
distance_table_walk.to_csv("data/maz_to_maz_dist_walk.csv", index=False)
distance_table_walk

distance_table_bike = distance_table[distance_table["DIST"] <= max_distance_for_nearby_mazs_bike]
distance_table_bike["TIME"] = distance_table["DIST"] * (60 * bike_speed)
distance_table_bike.to_csv("data/maz_to_maz_dist_bike.csv", index=False)
distance_table_bike

c:\programdata\anaconda3\envs\asimtestcli\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,OMAZ,DMAZ,DIST,TIME
0,1,1,0.12,72.0
1,1,2,0.37,222.0
2,1,3,0.57,342.0
3,1,4,0.34,204.0
4,1,5,0.70,420.0
...,...,...,...,...
620,25,21,1.44,864.0
621,25,22,0.84,504.0
622,25,23,1.03,618.0
623,25,24,0.50,300.0


### Remove other files not needed

In [8]:
if os.path.exists("data/mtc_asim.h5"):
    os.remove("data/mtc_asim.h5")
if os.path.exists("data/skims.omx"):
    os.remove("data/skims.omx")

In [9]:
#return to root folder
%cd ..

C:\projects\development\activitysim_rsg\notebooks


# Read inputs and write revised inputs for three_zone

In [10]:
%cd example_three_zone

C:\projects\development\activitysim_rsg\notebooks\example_three_zone


### Convert tazs to mazs

In [11]:
zones = pd.read_csv("data/land_use.csv")
zones["MAZ"] = zones["ZONE"]
zones["TAZ"] = zones["ZONE"]
zones.loc[zones["MAZ"].isin([1,2,3,4]),"TAZ"] = 2
zones.loc[zones["MAZ"].isin([13,14,15]),"TAZ"] = 14
zones = zones.drop(['ZONE'], axis=1)

zones.to_csv("data/land_use.csv", index=False)
zones

,DISTRICT,SD,COUNTY,TOTHH,HHPOP,TOTPOP,EMPRES,SFDU,MFDU,HHINCQ1,...,COLLFTE,COLLPTE,TOPOLOGY,TERMINAL,ZERO,hhlds,sftaz,gqpop,MAZ,TAZ
0,1,1,1,46,74,82,37,1,60,15,...,0.00000,0.00000,3,5.89564,0,46,1,8,1,2
1,1,1,1,134,214,240,107,5,147,57,...,0.00000,0.00000,1,5.84871,0,134,2,26,2,2
2,1,1,1,267,427,476,214,9,285,101,...,0.00000,0.00000,1,5.53231,0,267,3,49,3,2
3,1,1,1,151,239,253,117,6,210,52,...,0.00000,0.00000,2,5.64330,0,151,4,14,4,2
4,1,1,1,611,974,1069,476,22,671,223,...,72.14684,0.00000,1,5.52555,0,611,5,95,5,5
5,1,1,1,2240,3311,3963,2052,0,2406,843,...,0.00000,0.00000,1,5.00004,0,2240,6,652,6,6
6,1,1,1,3762,5561,6032,3375,0,4174,1489,...,0.00000,0.00000,1,5.35435,0,3762,7,471,7,7
7,1,1,1,4582,7565,9907,3594,19,4898,2704,...,0.00000,0.00000,2,4.64648,0,4582,8,2342,8,8
8,1,1,1,5545,9494,10171,4672,35,6032,2605,...,2035.58118,20.60887,2,5.22542,0,5545,9,677,9,9
9,1,1,1,5344,9205,9308,5137,5,5663,3825,...,690.54974,0.00000,3,4.73802,0,5344,10,103,10,10


### Put households in mazs instead of tazs

In [12]:
hhs = pd.read_csv("data/households.csv")
hhs["MAZ"] = hhs["TAZ"]
hhs = hhs.drop(['TAZ'], axis=1)
hhs.to_csv("data/households.csv", index=False)
hhs

,HHID,SERIALNO,PUMA5,income,PERSONS,HHT,UNITTYPE,NOC,BLDGSZ,TENURE,...,hschdriv,htypdwel,hownrent,hadnwst,hadwpst,hadkids,bucketBin,originalPUMA,hmultiunit,MAZ
0,2717868,2715386,2202,361000,2,1,0,0,9,1,...,0,2,1,0,0,0,3,2202,1,25
1,763899,5360279,2203,59220,1,4,0,0,9,3,...,0,2,2,0,0,0,4,2203,1,6
2,2222791,77132,2203,197000,2,2,0,0,9,1,...,0,2,1,0,0,1,5,2203,1,9
3,112477,3286812,2203,2200,1,6,0,0,8,3,...,0,2,2,0,0,0,7,2203,1,17
4,370491,6887183,2203,16500,3,1,0,1,8,3,...,0,2,2,0,0,0,7,2203,1,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,109218,3592966,2203,15000,1,4,0,0,8,3,...,0,2,2,0,0,0,7,2203,1,10
4996,570708,2418140,2202,13100,1,6,0,0,9,3,...,0,2,2,0,0,0,6,2202,1,23
4997,2762199,4016973,2203,0,1,0,2,0,0,0,...,0,2,2,0,0,0,2,2203,1,21
4998,2049372,965334,2203,103000,1,4,0,0,9,3,...,0,2,2,0,0,0,9,2203,1,18


### Create taz file, taps file, taz skims, and tap skims

In [13]:
skims_file = omx.open_file("data/skims.omx")
skims = skims_file.list_matrices()
num_zones = skims_file.shape()[0]
zone_labels = np.arange(1,(num_zones+1)) #seq from 1 to X

zones_to_drop = [1,3,4,13,15]
new_zone_labels = np.delete(zone_labels, np.array(zones_to_drop) - 1)
new_zone_indexes = new_zone_labels - 1 

tap_zone_labels = new_zone_labels + 10000

#create taz file
tazs = pd.DataFrame({"taz":new_zone_labels})
tazs.to_csv("data/tazs.csv", index=False)
print(tazs)

#create taps file
taps = pd.DataFrame({"tap":tap_zone_labels, "maz":(new_zone_labels)})
taps.to_csv("data/taps.csv", index=False)
print(taps)

# create skims
output_taz_skims_file = omx.open_file("data/taz_skims.omx", "w")
output_tap_skims_file = omx.open_file("data/tap_skims.omx", "w")

for a_skim in skims_file.list_matrices():
    
    if not (a_skim == "DISTBIKE" or a_skim == "DISTWALK"): #omit these since using MAZ to MAZ distances
        
        mode_code = a_skim[0:3]
        is_tap_mode = True if (mode_code == "DRV" or mode_code == "WLK") else False

        if not is_tap_mode:
            output_taz_skims_file[a_skim] = skims_file[a_skim][:][new_zone_indexes,:][:,new_zone_indexes]
            print("taz skim:", a_skim, ": shape", str(output_taz_skims_file.shape()))

        else:
            output_tap_skims_file[a_skim] = skims_file[a_skim][:][new_zone_indexes,:][:,new_zone_indexes]
            print("tap skim:", a_skim, ": shape", str(output_tap_skims_file.shape()))

output_taz_skims_file.create_mapping("taz", new_zone_labels)
output_tap_skims_file.create_mapping("tap", tap_zone_labels)
    
skims_file.close()
output_taz_skims_file.close()
output_tap_skims_file.close()

    taz
0     2
1     5
2     6
3     7
4     8
5     9
6    10
7    11
8    12
9    14
10   16
11   17
12   18
13   19
14   20
15   21
16   22
17   23
18   24
19   25
      tap  maz
0   10002    2
1   10005    5
2   10006    6
3   10007    7
4   10008    8
5   10009    9
6   10010   10
7   10011   11
8   10012   12
9   10014   14
10  10016   16
11  10017   17
12  10018   18
13  10019   19
14  10020   20
15  10021   21
16  10022   22
17  10023   23
18  10024   24
19  10025   25
taz skim: DIST : shape (20, 20)
tap skim: DRV_COM_WLK_BOARDS__AM : shape (20, 20)
tap skim: DRV_COM_WLK_BOARDS__EA : shape (20, 20)
tap skim: DRV_COM_WLK_BOARDS__EV : shape (20, 20)
tap skim: DRV_COM_WLK_BOARDS__MD : shape (20, 20)
tap skim: DRV_COM_WLK_BOARDS__PM : shape (20, 20)
tap skim: DRV_COM_WLK_DDIST__AM : shape (20, 20)
tap skim: DRV_COM_WLK_DDIST__EA : shape (20, 20)
tap skim: DRV_COM_WLK_DDIST__EV : shape (20, 20)
tap skim: DRV_COM_WLK_DDIST__MD : shape (20, 20)
tap skim: DRV_COM_WLK_DDIST__PM : shape

tap skim: DRV_LOC_WLK_FAR__MD : shape (20, 20)
tap skim: DRV_LOC_WLK_FAR__PM : shape (20, 20)
tap skim: DRV_LOC_WLK_IWAIT__AM : shape (20, 20)
tap skim: DRV_LOC_WLK_IWAIT__EA : shape (20, 20)
tap skim: DRV_LOC_WLK_IWAIT__EV : shape (20, 20)
tap skim: DRV_LOC_WLK_IWAIT__MD : shape (20, 20)
tap skim: DRV_LOC_WLK_IWAIT__PM : shape (20, 20)
tap skim: DRV_LOC_WLK_TOTIVT__AM : shape (20, 20)
tap skim: DRV_LOC_WLK_TOTIVT__EA : shape (20, 20)
tap skim: DRV_LOC_WLK_TOTIVT__EV : shape (20, 20)
tap skim: DRV_LOC_WLK_TOTIVT__MD : shape (20, 20)
tap skim: DRV_LOC_WLK_TOTIVT__PM : shape (20, 20)
tap skim: DRV_LOC_WLK_WAIT__AM : shape (20, 20)
tap skim: DRV_LOC_WLK_WAIT__EA : shape (20, 20)
tap skim: DRV_LOC_WLK_WAIT__EV : shape (20, 20)
tap skim: DRV_LOC_WLK_WAIT__MD : shape (20, 20)
tap skim: DRV_LOC_WLK_WAIT__PM : shape (20, 20)
tap skim: DRV_LOC_WLK_WAUX__AM : shape (20, 20)
tap skim: DRV_LOC_WLK_WAUX__EA : shape (20, 20)
tap skim: DRV_LOC_WLK_WAUX__EV : shape (20, 20)
tap skim: DRV_LOC_WLK_WAUX_

tap skim: WLK_COM_DRV_IWAIT__EV : shape (20, 20)
tap skim: WLK_COM_DRV_IWAIT__MD : shape (20, 20)
tap skim: WLK_COM_DRV_IWAIT__PM : shape (20, 20)
tap skim: WLK_COM_DRV_KEYIVT__AM : shape (20, 20)
tap skim: WLK_COM_DRV_KEYIVT__EA : shape (20, 20)
tap skim: WLK_COM_DRV_KEYIVT__EV : shape (20, 20)
tap skim: WLK_COM_DRV_KEYIVT__MD : shape (20, 20)
tap skim: WLK_COM_DRV_KEYIVT__PM : shape (20, 20)
tap skim: WLK_COM_DRV_TOTIVT__AM : shape (20, 20)
tap skim: WLK_COM_DRV_TOTIVT__EA : shape (20, 20)
tap skim: WLK_COM_DRV_TOTIVT__EV : shape (20, 20)
tap skim: WLK_COM_DRV_TOTIVT__MD : shape (20, 20)
tap skim: WLK_COM_DRV_TOTIVT__PM : shape (20, 20)
tap skim: WLK_COM_DRV_WAIT__AM : shape (20, 20)
tap skim: WLK_COM_DRV_WAIT__EA : shape (20, 20)
tap skim: WLK_COM_DRV_WAIT__EV : shape (20, 20)
tap skim: WLK_COM_DRV_WAIT__MD : shape (20, 20)
tap skim: WLK_COM_DRV_WAIT__PM : shape (20, 20)
tap skim: WLK_COM_DRV_WAUX__AM : shape (20, 20)
tap skim: WLK_COM_DRV_WAUX__EA : shape (20, 20)
tap skim: WLK_COM

tap skim: WLK_HVY_DRV_FAR__AM : shape (20, 20)
tap skim: WLK_HVY_DRV_FAR__EA : shape (20, 20)
tap skim: WLK_HVY_DRV_FAR__EV : shape (20, 20)
tap skim: WLK_HVY_DRV_FAR__MD : shape (20, 20)
tap skim: WLK_HVY_DRV_FAR__PM : shape (20, 20)
tap skim: WLK_HVY_DRV_IWAIT__AM : shape (20, 20)
tap skim: WLK_HVY_DRV_IWAIT__EA : shape (20, 20)
tap skim: WLK_HVY_DRV_IWAIT__EV : shape (20, 20)
tap skim: WLK_HVY_DRV_IWAIT__MD : shape (20, 20)
tap skim: WLK_HVY_DRV_IWAIT__PM : shape (20, 20)
tap skim: WLK_HVY_DRV_KEYIVT__AM : shape (20, 20)
tap skim: WLK_HVY_DRV_KEYIVT__EA : shape (20, 20)
tap skim: WLK_HVY_DRV_KEYIVT__EV : shape (20, 20)
tap skim: WLK_HVY_DRV_KEYIVT__MD : shape (20, 20)
tap skim: WLK_HVY_DRV_KEYIVT__PM : shape (20, 20)
tap skim: WLK_HVY_DRV_TOTIVT__AM : shape (20, 20)
tap skim: WLK_HVY_DRV_TOTIVT__EA : shape (20, 20)
tap skim: WLK_HVY_DRV_TOTIVT__EV : shape (20, 20)
tap skim: WLK_HVY_DRV_TOTIVT__MD : shape (20, 20)
tap skim: WLK_HVY_DRV_TOTIVT__PM : shape (20, 20)
tap skim: WLK_HVY_DR

tap skim: WLK_LRF_DRV_WAUX__EA : shape (20, 20)
tap skim: WLK_LRF_DRV_WAUX__EV : shape (20, 20)
tap skim: WLK_LRF_DRV_WAUX__MD : shape (20, 20)
tap skim: WLK_LRF_DRV_WAUX__PM : shape (20, 20)
tap skim: WLK_LRF_DRV_XWAIT__AM : shape (20, 20)
tap skim: WLK_LRF_DRV_XWAIT__EA : shape (20, 20)
tap skim: WLK_LRF_DRV_XWAIT__EV : shape (20, 20)
tap skim: WLK_LRF_DRV_XWAIT__MD : shape (20, 20)
tap skim: WLK_LRF_DRV_XWAIT__PM : shape (20, 20)
tap skim: WLK_LRF_WLK_BOARDS__AM : shape (20, 20)
tap skim: WLK_LRF_WLK_BOARDS__EA : shape (20, 20)
tap skim: WLK_LRF_WLK_BOARDS__EV : shape (20, 20)
tap skim: WLK_LRF_WLK_BOARDS__MD : shape (20, 20)
tap skim: WLK_LRF_WLK_BOARDS__PM : shape (20, 20)
tap skim: WLK_LRF_WLK_FAR__AM : shape (20, 20)
tap skim: WLK_LRF_WLK_FAR__EA : shape (20, 20)
tap skim: WLK_LRF_WLK_FAR__EV : shape (20, 20)
tap skim: WLK_LRF_WLK_FAR__MD : shape (20, 20)
tap skim: WLK_LRF_WLK_FAR__PM : shape (20, 20)
tap skim: WLK_LRF_WLK_FERRYIVT__AM : shape (20, 20)
tap skim: WLK_LRF_WLK_FERR

### Create maz to maz distance file by mode

In [14]:
skims_file = omx.open_file("data/skims.omx")

distance_table = pd.DataFrame(skims_file['DIST']).unstack()

skims_file.close()

walk_speed = 3
bike_speed = 10
max_distance_for_nearby_mazs_walk = 1.0
max_distance_for_nearby_mazs_bike = 5.0

distance_table = distance_table.reset_index()
distance_table.columns = ["OMAZ","DMAZ","DIST"]
distance_table["OMAZ"] = distance_table["OMAZ"] + 1 #seq from 1 to X
distance_table["DMAZ"] = distance_table["DMAZ"] + 1 #seq from 1 to X

distance_table_walk = distance_table[distance_table["DIST"] <= max_distance_for_nearby_mazs_walk]
distance_table_walk["TIME"] = distance_table["DIST"] * (60 / walk_speed)
distance_table_walk.to_csv("data/maz_to_maz_dist_walk.csv", index=False)
distance_table_walk

distance_table_bike = distance_table[distance_table["DIST"] <= max_distance_for_nearby_mazs_bike]
distance_table_bike["TIME"] = distance_table["DIST"] * (60 * bike_speed)
distance_table_bike.to_csv("data/maz_to_maz_dist_bike.csv", index=False)
distance_table_bike

c:\programdata\anaconda3\envs\asimtestcli\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,OMAZ,DMAZ,DIST,TIME
0,1,1,0.12,72.0
1,1,2,0.37,222.0
2,1,3,0.57,342.0
3,1,4,0.34,204.0
4,1,5,0.70,420.0
...,...,...,...,...
620,25,21,1.44,864.0
621,25,22,0.84,504.0
622,25,23,1.03,618.0
623,25,24,0.50,300.0


### Create maz to tap distance file by mode

In [15]:
skims_file = omx.open_file("data/skims.omx")

distance_table = pd.DataFrame(skims_file['DIST']).unstack()

skims_file.close()

walk_speed = 3
bike_speed = 10
drive_speed = 25
max_distance_for_nearby_taps_walk = 1.0
max_distance_for_nearby_taps_bike = 5.0
max_distance_for_nearby_taps_drive = 10.0

distance_table = distance_table.reset_index()
distance_table.columns = ["MAZ","TAP","DIST"]
distance_table["MAZ"] = distance_table["MAZ"] + 1 #seq from 1 to X
distance_table["TAP"] = distance_table["TAP"] + 1 #seq from 1 to X
distance_table["TAP"] = distance_table["TAP"] + 10000
distance_table = distance_table[distance_table["TAP"].isin(tap_zone_labels)]

distance_table_walk = distance_table[distance_table["DIST"] <= max_distance_for_nearby_taps_walk]
distance_table_walk["TIME"] = distance_table["DIST"] * (60 / walk_speed)
distance_table_walk.to_csv("data/maz_to_tap_dist_walk.csv", index=False)
distance_table_walk

distance_table_bike = distance_table[distance_table["DIST"] <= max_distance_for_nearby_taps_bike]
distance_table_bike["TIME"] = distance_table["DIST"] * (60 * bike_speed)
distance_table_bike.to_csv("data/maz_to_tap_dist_bike.csv", index=False)
distance_table_bike

distance_table_drive = distance_table[distance_table["DIST"] <= max_distance_for_nearby_taps_drive]
distance_table_drive["TIME"] = distance_table["DIST"] * (60 * drive_speed)
distance_table_drive.to_csv("data/maz_to_tap_dist_drive.csv", index=False)
distance_table_drive

c:\programdata\anaconda3\envs\asimtestcli\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,MAZ,TAP,DIST,TIME
1,1,10002,0.37,555.0
4,1,10005,0.70,1050.0
5,1,10006,0.97,1455.0
6,1,10007,1.02,1530.0
7,1,10008,1.05,1575.0
...,...,...,...,...
620,25,10021,1.44,2160.0
621,25,10022,0.84,1260.0
622,25,10023,1.03,1545.0
623,25,10024,0.50,750.0


### Remove other files not needed

In [16]:
if os.path.exists("data/mtc_asim.h5"):
    os.remove("data/mtc_asim.h5")
if os.path.exists("data/skims.omx"):
    os.remove("data/skims.omx")